# Measuring uptake of inhaled budesonide prescription guidance following COVID-19 infection

## Setup, data loading and prepreprocessing

In [ ]:
import pandas as pd

In [ ]:
## low count suppression function
def suppress(df_to_suppress,columns,n=10):
    for c in columns:
        if not c in df_to_suppress.columns:
            raise ValueError(f'column {c} not in dataframe')

    df = df_to_suppress
    for c in df.columns[df.columns.isin(columns)]:
        df[f'{c}'] = df[f'{c}'].astype(str)
        df[f'{c}'] = df[f'{c}'].apply(lambda x: '<10' if (int(x)>0 and int(x)<n ) else x)
    return df

In [ ]:
df = pd.read_csv('../output/input.csv')

In [ ]:
print(f'Total input row count: {len(df.index)}')

In [ ]:
#include only where positive test date is populated
df = df[df.first_positive_test_date.notna()]
print(f'Patients with positive test: {len(df.index)}')

In [ ]:
#add cohort label [over 65s, over 55s with shielding flag, over 55s with non-shielding flag]
df['cohort'] = df.apply(lambda x: x.age_band if x.age_band == '65_plus' else str(x.age_band) + ('H' if x.primis_shield==1 else 'L'),axis=1)
df_cohort_counts = df.groupby('cohort').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"cohort":"Cohort"})
suppressed_df_cohort_counts = suppress(df_cohort_counts,['Patient count'])
suppressed_df_cohort_counts.style.hide_index()

## COVID-19 Test Types

In [ ]:
#exclude LFT tests, include unknowns
df.first_positive_test_type= df.first_positive_test_type.fillna('Unknown')
df = df[df.first_positive_test_type!="LFT_Only"]
print(f'Patients with non-LFT_Only test: {len(df.index)}')

In [ ]:
df_test_counts = df.groupby('first_positive_test_type').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"first_positive_test_type":"Type of first recorded positive test"})
supressed_test_type_counts = suppress(df_test_counts,['Patient count'])
supressed_test_type_counts.style.hide_index()

## Budesonide prescriptions by cohort

In [ ]:
supressed_bd_prescriptions = suppress(df.groupby('budesonide_prescription').count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),["Count of patients"])
supressed_bd_prescriptions.style.hide_index()

In [ ]:
supressed_bd_prescriptions_by_cohort = suppress(df.groupby(['cohort','budesonide_prescription']).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),['Count of patients'])
supressed_bd_prescriptions_by_cohort.style.hide_index()